Давайте решим следующую задачу.<br>
Необходимо написать робота, который будет скачивать новости с сайта Лента.Ру и фильтровать их в зависимости от интересов пользователя. От пользователя требуется отмечать интересующие его новости, по которым система будет выделять области его интересов.<br>


Начнем с загрузки новостей. Для этого нам потребуется метод requests.get(url). Библиотека requests предоставляет серьезные возможности для загрузки информации из Интернет. Метод get получает URL стараницы и возвращает ее содержимое. В нашем случае результат будет получаться в формате html. <br>
Загрузим необходимые библиотеки.

In [1]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.

Теперь попробуем загрузить страницу новостей.

In [2]:
# Для пробы получаем первую страницу сайта.
requests.get("http://lenta.ru/")

<Response [200]>

Метод <i>requests.get()</i> возвращает объект Response, который содержит большое количество различной информации о загруженной (или незагруженной) странице. В краткой форме отображается только результат выполения запроса. В нашем случае это 200, нет ошибки.<br> 
Посмотрим что результат содержит еще.

In [3]:
resp = requests.get("https://lenta.ru/news/2018/08/24/clon/")
dir(resp)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [4]:
%%time
# %%time - Магия Jupyter - замеряет время выполнения ячейки. Должно быть первой строчкой в ячейке.
resp = requests.get("https://lenta.ru/news/2018/08/24/clon/")
print("cookies:", resp.cookies)
print("time to download:", resp.elapsed)
print("page encoding", resp.encoding)
print("Server response: ", resp.status_code)
print("Is everything ok? ", resp.ok)
print("Page's URL: ", resp.url)

cookies: <RequestsCookieJar[<Cookie is_mobile=0 for .lenta.ru/>, <Cookie lid=vAsAABRDQGMrWHKpAQ1RAwB= for .lenta.ru/>, <Cookie lids=4820582B10C5AA78 for .lenta.ru/>]>
time to download: 0:00:00.133192
page encoding utf-8
Server response:  200
Is everything ok?  True
Page's URL:  https://lenta.ru/news/2018/08/24/clon/
CPU times: user 15.4 ms, sys: 4.39 ms, total: 19.7 ms
Wall time: 148 ms


Но самое для нас интересное хранится в поле <i>text</i>, которое содержит собственно текст html-страницы.

In [5]:
#Берем первые 1000 символов новости.
resp.text[:1000]

'<!DOCTYPE html><html lang="ru"><head><title>В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru</title><meta charset="utf-8" /><meta content="#292929" name="theme-color" /><link href="https://m.lenta.ru/news/2018/08/24/clon/" media="only screen and (max-width: 640px)" rel="alternate" /><link href="https://lenta.ru/rss/google-newsstand/main/" rel="alternate" type="application/rss+xml" /><link href="https://lenta.ru/news/2018/08/24/clon/" rel="canonical" /><link href="/manifest.json" rel="manifest" /><link rel="shortcut icon" type="image/x-icon" href="https://icdn.lenta.ru/favicon.ico" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-256x256.png" size="256x256" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-192x192.png" size="192x192" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-152x152.png" size="

Количество служебной информации в странице явно превышает объем текста новости. У нас есть два пути: либо использовать библиотеку BeautyfulSoup для получения текста статьи, либо получить текст с использованием регулярных выражений.

Опробуем первый путь. Документация на библиотеку BeautyfulSoup находится <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">здесь</a>.

В ячейке ниже мы создаем объект BeautifulSoup, передаем в него текст html-страницы и сообщаем, что разбирать его надо при помощи библиотеки `html5lib`. Далее просим отдать текст страницы без html-тегов.

In [6]:
BeautifulSoup(resp.text, "html5lib").get_text()

'В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru{"@context":"http://schema.org","@type":"NewsArticle","headline":"В Сибири нашли подходящих для клонирования древних животных","description":"Палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Подобные находки имеют большое количество сохранившейся ДНК, которая подходит для клонирования.","name":"В Сибири нашли подходящих для клонирования древних животных","url":"https://lenta.ru/news/2018/08/24/clon/","mainEntityOfPage":{"@type":"WebPage","@id":"https://lenta.ru/news/2018/08/24/clon/"},"associatedMedia":"","thumbnailUrl":"https://icdn.lenta.ru/images/2018/08/24/13/20180824130540171/detail_fb4ef24e26e8448a2a23747f000a4489.jpg","dateCreated":"2018-08-24T15:22:00+03:00","datePublished":"2018-08-24T15:22:00+03:00","dateModified":"20

Да, убрать html-теги получилось. Но их содержимое осталось, в том числе и скрипты.<br>
Опробуем другой путь. Весь текст обычно оформляется тегом параграфа - &lt;p&gt;. Выберем весь текст из этих тегов. Заодно выберем и заголовок статьи, оформленный при помощи <h1>

In [7]:
# Получили объект BeautifulSoup и скормили ему текст страницы.
bs = BeautifulSoup(resp.text, "html5lib") 
# Вот таким образом можно попросить отдать первый тег, отмеченный как h1. Вместо h1 можно написать любой другой тег.
title = bs.h1.text
# Получаем все параграфы (тег p), берем их текст без тегов и склеиваем в один текст.
text = " ".join([p.text for p in bs.find_all("p")])
print(title, "\n-----\n", text)

В Сибири нашли подходящих для клонирования древних животных 
-----
 Фото: AP Российские палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Об этом сообщается в пресс-релизе на Phys.org. Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Таким образом, находка является потенциально пригодной для клонирования животного. У найденного ископаемого, относящегося к вымершему виду Equus lenensis, сохранились кожа, шерсть, копыта, хвост и внутренние органы. Возраст жеребенка на момент смерти составлял примерно 2-3 месяца. Причиной смерти, вероятно, является попадание в какую-то «ловушку» естественного происхождения, поскольку видимых повреждений на теле не было. У трупа были взяты образцы шерсти и биологических жидкостей для тщательного генетического анализа. По словам исследователей, на данный момент это самые хорошо сохранившиеся из всех останков древних лошадей. В 2015 году в Я

Теперь напишем функцию, которая выгружает все новости за сутки. <br>
Обратим внимание, что для сайта Lenta.ru можно написать адрес в формате lenta.ru/ГГГГ/ММ/ДД/ (год, месяц, день) и получить все новости за этот день. Попробуем получить все адреса с такой страницы.

In [24]:
# Идем на страницу, получаем ее текст, отдаем в BeautifulSoup, ищем все теги ссылок - а.
BeautifulSoup(requests.get("http://lenta.ru/2018/08/24/").text, "html5lib").find_all("a")[:20]

[<a class="menu__nav-link _is-extra" href="/">Главное</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/russia/">Россия</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/world/">Мир</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/ussr/">Бывший СССР</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/economics/">Экономика</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/forces/">Силовые структуры</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/science/">Наука и техника</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/culture/">Культура</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/sport/">Спорт</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/media/">Интернет и СМИ</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/style/">Ценности </a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/travel/">Путешествия</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/life/">Из жизни</a>,
 <a class="menu__nav-lin

Кажется, это опять немного не то, что нам нужно. Мы получили все ссылки, находящиеся на боковом меню, ссылки на события сегодняшнего дня и другие ненужные нам вещи. <br>
Смотрим в содержимое html-страницы и обращаем внимание, что все интересные нам ссылки оформлены как заголовки третьего уровня - &lt;h3&gt;. Извлечем все такие фрагменты, а потом извлечем собственно адреса, помеченные атрибутом href тега &lt;a&gt;.

In [25]:
# Теперь выделим только то, что взято в тег h3.
h3s = BeautifulSoup(requests.get("http://lenta.ru/2018/08/24/").text, "html5lib").find_all("h3")

BeautifulSoup хранит именно дерево HTML-страницы, поэтому на самом деле он возвращает узлы дерева. И как у каждого уважающего себя узла, у него есть ссылка на родительский узел. В нашем случае он будет хранить ссылку на страницу.

In [21]:
h3s[10].parent.name

'a'

In [26]:
# Формируем список ссылок. Для этого берем первую (кстати, единственную) ссылку из каждого выделенного
# фрагмента, у нее берем значение параметра href. Так как ссылки внутренние, добавляем к ним адрес сайта.
links = ["http://lenta.ru" + l.parent["href"] for l in h3s if l.parent.name == 'a']
print(links)

['http://lenta.ru/news/2018/08/24/agutin/', 'http://lenta.ru/news/2018/08/24/f35/', 'http://lenta.ru/news/2018/08/24/stop/', 'http://lenta.ru/news/2018/08/24/video_napadenie/', 'http://lenta.ru/news/2018/08/24/king/', 'http://lenta.ru/news/2018/08/24/desire/', 'http://lenta.ru/news/2018/08/24/parade/', 'http://lenta.ru/news/2018/08/24/reson/', 'http://lenta.ru/news/2018/08/24/su27/', 'http://lenta.ru/news/2018/08/23/bomb/', 'http://lenta.ru/news/2018/08/24/jail/', 'http://lenta.ru/news/2018/08/24/27/', 'http://lenta.ru/news/2018/08/24/burn/', 'http://lenta.ru/news/2018/08/24/xhamster/', 'http://lenta.ru/news/2018/08/24/troll/', 'http://lenta.ru/news/2018/08/24/python/', 'http://lenta.ru/news/2018/08/24/drink/', 'http://lenta.ru/news/2018/08/24/s400/', 'http://lenta.ru/news/2018/08/23/var_cl/', 'http://lenta.ru/news/2018/08/24/toshnota/', 'http://lenta.ru/news/2018/08/24/sami/', 'http://lenta.ru/news/2018/08/24/tesla/', 'http://lenta.ru/news/2018/08/24/bear/']


Если теперь написать функцию, которая будет перебирать все адреса и получать из них тексты новостей, то мы получим все новости за определенные сутки. Заодно надо обратить внимание на кнопку `"Дальше"`, которая намекает, что нам показывают не все новости за сутки. Но это мы сделаем в следующей жизни, а пока просто оформим код загрузки статьи в виде функции.

In [27]:
# Загрузка статьи по URL.
def getOneLentaArticle(url):
    """ getLentaArticle gets the body of an article from Lenta.ru"""
    # Получает текст страницы.
    resp = requests.get(url)
    # Загружаем текст в объект типа BeautifulSoup.
    bs = BeautifulSoup(resp.text, "html5lib") 
    # Получаем заголовок статьи.
    aTitle = bs.h1.text.replace("\xa0", " ")
    # Получаем текст статьи.
    anArticle = BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
    # Возвращаем кортеж из заголовка и текста статьи.
    return aTitle, anArticle


### XPath
[Ссылка 1](https://habr.com/ru/post/526774/)

[Ссылка 2](https://habr.com/ru/post/464897/)

Альтернативной библиотекой является XPath, который позволяет задать шаблон для пути от корня XML-дерева к интересующей нас вершине.

- . - корень XML-дерева
- / - переход на один уровень ниже.
- // - переход на ноль или больше уровней вниз.
- \* - любая вершина.
- xyz - название вершины.
- [@feature] - вершина с параметром feature.
- [@feature='111'] - вершина с параметром feature, равным "111".
- xyz[n] - n-ый потомок вершины xyz.

А теперь давайте посмотрим как мы можем при помощи XPath обрабатывать HTML-документы.

In [28]:
from lxml import html

In [29]:
page = requests.get('https://lenta.ru/news/2021/02/27/apple_effect/')

In [38]:
tree = html.fromstring(page.text)
print(tree.xpath(".//h1")[0].text_content())
print(tree.xpath(".//time[contains(@class, 'topic-header__time')]")[0].text_content().strip())
print(tree.xpath(".//div[contains(@class, 'topic-authors')]")[0].text_content().strip(), '\n')
print(tree.xpath(".//meta[@property='og:description']")[0].get("content"), '\n')

for p in tree.xpath(".//div[contains(@class, '_news')]//p[contains(@class, 'topic-body__content-text')]"):
    print(p.text_content())

Обнаружен неожиданный эффект от употребления яблок
15:35, 27 февраля 2021
Соня Кошечкина 

Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний  обнаружили неожиданный эффект от употребления яблок. Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. 

Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.
Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.
По словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффек

А теперь та же страница, но через BeautyfulSoup.

In [42]:
souped = BeautifulSoup(page.text)

print(souped("h1")[0].get_text())
print(souped.find_all("time", attrs={'class': 'topic-header__time'})[0].get_text().strip())
print(souped.find_all("div", attrs={'class': 'topic-authors'})[0].get_text())
print(souped.find_all("meta", attrs={'property': 'og:description'})[0]["content"], '\n')

for p in souped.find_all("div", attrs={'class': '_news'})[0]("p", attrs={'class': 'topic-body__content-text'}): 
    print(p.get_text())

Обнаружен неожиданный эффект от употребления яблок
15:35, 27 февраля 2021
Соня Кошечкина
Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний  обнаружили неожиданный эффект от употребления яблок. Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. 

Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.
Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.
По словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект,

## @dataclass

Иногда нам необходимо создать класс, который будет содержать в себе только данные, но не будет содержать в себе методов работы с этими данными. Для этого существует декоратор `dataclass` из библиотеки `dataclasses`.

Их удобство заключается в том, что можно просто описать поля, входящие в этот класса, и все объекты будут создаваться с этими полями. При этом обязательно надо указывать тип атрибута. (Но можно указать `Any` из модуля `typing`.)

При необходимости можно присвоить атрибутам значения по умолчанию. Но следует иметь в виду, что сперва идут все поля без значений по умолчанию, а потом все с присваиваемыми значениями.

Заметим, что в таких классах могут быть и методы. Просто иногда проще описать такой класс и ничего в него не добавлять, а значения по умолчанию пусть берутся из описания.

Более подробно про них можно посмотреть [здесь](https://habr.com/ru/post/415829/) и [здесь](https://docs.python.org/3/library/dataclasses.html)


In [44]:
from dataclasses import dataclass

In [60]:
@dataclass
class LentaArticle:
    title: str
    text: str
    description: str
    time: str = "00:00"
    author: str = "No author"
        


Теперь напишем функцию, которая будет возвращать объект новости, а не будет хранить атрибуты одной сущности в разных местах.

In [63]:
def get_lenta_article(url: str) -> LentaArticle:
    page = requests.get(url)
    tree = html.fromstring(page.text)
    ttl = tree.xpath(".//h1")[0].text_content()
    dscrptn = tree.xpath(".//meta[@property='og:description']")[0].get("content")

    txt = '\n'.join([p.text_content() for p in 
             tree.xpath(".//div[contains(@class, '_news')]//p[contains(@class, 'topic-body__content-text')]")]
                    )
    
    article = LentaArticle(ttl, txt, dscrptn)
    article.time = tree.xpath(".//time[contains(@class, 'topic-header__time')]")[0].text_content().strip()
    article.author = tree.xpath(".//div[contains(@class, 'topic-authors')]")[0].text_content().strip()
    return article

get_lenta_article('https://lenta.ru/news/2021/02/27/apple_effect/')

LentaArticle(title='Обнаружен неожиданный эффект от употребления яблок', text='Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.\nОпыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.\nПо словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект, что и физическая активность, которая также стимулирует нейрогенез.\nРанее ученые из Технологического университета австрийского Граца выяснили, что большинство людей неправильно едят яблоки. Исследователи утверждают, что до 90 процентов полезных веществ сосредоточены в сердцевине этого фрукта, и п

Вообще-то, как-то не очень красиво. А давайте, раз уж можно заводить функции, создадим конструктор, в который будут передаваться значения в удобном для нас порядке. А заодно заведем метод `__repr__`.

In [70]:
@dataclass()
class LentaArticle:
    title: str
    text: str
    description: str
    time: str = "00:00"
    author: str = "No author"
        
    def __init__(self: 'LentaArticle', _title: str, _author: str, _description: str,
                 _time: str, _text: str):
        self.title = _title
        self.author = _author
        self.description = _description
        self.time = _time
        self.text = _text
        
    def __repr__(self: 'LentaArticle') -> str:
        return f"""LentaArticle(title={self.title[:60]}\nauthor={self.author}\n"""\
               f"""time={self.time}\n{self.text[:100]}..."""
        
        
def get_lenta_article(url: str) -> LentaArticle:
    page = requests.get(url)
    tree = html.fromstring(page.text)
    article = LentaArticle(
            tree.xpath(".//h1")[0].text_content(),
            tree.xpath(".//div[contains(@class, 'topic-authors')]")[0].text_content().strip(),
            tree.xpath(".//meta[@property='og:description']")[0].get("content"),
            tree.xpath(".//time[contains(@class, 'topic-header__time')]")[0].text_content().strip(), 
            '\n'.join([p.text_content() for p in 
                tree.xpath(".//div[contains(@class, '_news')]//p[contains(@class, 'topic-body__content-text')]")]
                    )
           )
    
    return article

get_lenta_article('https://lenta.ru/news/2021/02/27/apple_effect/')

LentaArticle(title=Обнаружен неожиданный эффект от употребления яблок
author=Соня Кошечкина
time=15:35, 27 февраля 2021
Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неож...

Декоратор `@dataclass` обладает целым рядом параметров, помогающих проще решать некоторые задачи. 

`frozen=True` - в объекты нельзя будет добавлять новые атрибуты.  
`init, repr, eq, order =True` - заводят соответствующие функции по умолчанию: конструктор, представления, эквивалентности, сравнения.

In [71]:
@dataclass(frozen=True)
class LentaArticleFrozen:
    title: str = ""
    text: str = ""
    description: str = ""
    time: str = "00:00"
    author: str = "No author"
        
aaa = LentaArticleFrozen()
aaa.newOne = 1

FrozenInstanceError: cannot assign to field 'newOne'

Запустим вот такой код. И что мы увидим?

In [72]:
@dataclass()
class LentaArticleMAuthors:
    title: str
    text: str
    description: str
    time: str = "00:00"
    author: str = []

ValueError: mutable default <class 'list'> for field author is not allowed: use default_factory

1. Никто не следит за типами значений.
2. Присваивать мутабельные типы нельзя. Предлагают использовать `default_factory`.

In [75]:
from dataclasses import field

@dataclass()
class LentaArticleMAuthors:
    title: str
    text: str
    description: str
    time: str = "00:00"
    author: list[str] = field(default_factory=list)